In [1]:
import numpy as np
import pandas as pd

이 문서에서는 어떻게 단변량, 다변량 시계열 예측 문제를<br>
지도학습으로 변환할 수 있는지에 대해 공부한다.<br>
이 문서에서 다룰 내용은 다음과 같다.<br>
<ol>
    <li>시계열 데이터셋을 지도학습 데이터셋으로 변환할 수 있는 함수</li>
    <li>기계학습을 위한 단변량 시계열 데이터 변환</li>
    <li>기계학습을 위한 다변량 시계열 데이터 변환</li>
</ol>

시계열 vs 지도학습<br><br>
시계열 데이터는 단순히 순차적인 순서를 갖는 데이터셋이다.<br>

In [17]:
# ex)
ex_time = pd.DataFrame([1, 2, 3, 4, 5, None, None], columns=['t'])
ex_time

,t
0,1.0
1,2.0
2,3.0
3,4.0
4,5.0
5,NaN
6,NaN


지도학습 데이터셋은 입력과 출력 쌍으로 구성된다.

In [3]:
# ex)
ex_sup = pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]})
ex_sup

,x,y
0,1,4
1,2,5
2,3,6


Pandas shift function<br><br>
shift는 시계열 데이터를 지도학습 데이터셋으로 변환할 수 있는 핵심 함수이다.<br>
shift는 원본데이터를 복사해서 앞으로 밀거나 뒤로 당겨준다.<br>
lag 관측치에 대한 새로운 컬럼을 생성해야 한다. 이 컬럼은 학습 능력을 향상시킬 수 있다.

In [7]:
ex_time['t-1'] = ex_time.t.shift(1)
ex_time

,t,t-1
0,1,NaN
1,2,1.0
2,3,2.0
3,4,3.0
4,5,4.0


series_to_supervised<br><br>
파라미터<br>
1. data: 2차원의 넘파이 배열(시계열 관측치)<br>
2. n_in: X의 lag 관측치 개수. 기본값은 1<br>
3. n_out: y의 개수. 기본값은 1<br>
4. dropnan: NaN값을 갖는 로우를 버릴지 말지 선택. 기본값은 True<br><br>
반환값: 지도학습을 위한 프레임.

In [22]:
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=False):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]

    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]

    agg = concat(cols, axis=1)
    agg.columns = names

    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [26]:
series_to_supervised(ex_time, 3, 1)

,var1(t-3),var1(t-2),var1(t-1),var1(t)
0,NaN,NaN,NaN,1.0
1,NaN,NaN,1.0,2.0
2,NaN,1.0,2.0,3.0
3,1.0,2.0,3.0,4.0
4,2.0,3.0,4.0,5.0
5,3.0,4.0,5.0,NaN
6,4.0,5.0,NaN,NaN
